In [6]:
import gensim
import json
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image
from IPython.core.display import HTML 
morph = MorphAnalyzer()
import numpy as np

Лемматизация и токенизация, тут код без изменений:

In [7]:
stops = set(stopwords.words('russian'))

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

In [9]:
texts = open('wiki_data.txt', encoding='utf-8').read().splitlines()[:10000]

In [11]:
texts = opt_normalize([tokenize(text.lower()) for text in texts], 30000)

Добавляем нграммы:

In [58]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.6) # при значении в 0.4 нграммы были на мой вкус недостаточно "устойчивые"
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

In [59]:
ngrammed_texts[0]

['нижегородский',
 '—',
 'сельский',
 'посёлок',
 'район',
 'нижегородский',
 'область',
 'входить',
 'состав',
 'расположить',
 '12,5',
 'километр',
 'юг',
 'село',
 '1',
 'километр',
 'запасть',
 'город',
 'право',
 'берег_река',
 'правый',
 'приток',
 'река',
 'сатис',
 'окружить',
 'смешанный',
 'леса',
 'соединить',
 'дорогой',
 'посёлок',
 '1,5',
 'километр',
 'дорога',
 'посёлок',
 'сатис',
 '3,5',
 'километр',
 'название',
 'являться',
 'сугубо',
 'официальный',
 'местный',
 'население',
 'использовать',
 'исключительно',
 'название',
 '—',
 'употребляться_языковой',
 'оборот',
 'ранее',
 'использовать',
 'название',
 '—',
 '1920-ха',
 'год',
 'переселенец',
 'соседний',
 'село',
 'аламасовый',
 'расположить',
 'соответственно',
 '8',
 '14',
 'километр',
 'запасть',
 'посёлок',
 'жить',
 'рабочий',
 'совхоз',
 'центр',
 'посёлок',
 'сатис',
 'возле',
 'посёлок',
 'расположить',
 'активно',
 'камень',
 'настоящее_время',
 'официально',
 'данные',
 '1978',
 'год',
 'посёлок',
 'н

# LDA

Составляем словарь. Удалять стоп-слова тут вроде не нужно, потому что мы от них избавились еще на этапе нормализации.

In [61]:
dictionary = gensim.corpora.Dictionary(texts)

In [64]:
dictionary.filter_extremes(no_above=0.5, no_below=20)
dictionary.compactify()

In [66]:
print(dictionary)

Dictionary(5782 unique tokens: ['1', '1,2', '1,5', '12', '14']...)


Преобразуем тексты в мешки слов. 

In [81]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [91]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
corpus = tfidf[corpus]

Теперь переходим к обучению. Обучим несколько моделей с разными параметрами и для каждой будем выводить полученные темы и считать перплексию.

Сначала увеличим количество тем до 200.

In [82]:
%%time
lda_1 = gensim.models.LdaMulticore(corpus, 200, id2word=dictionary, passes=10)

Wall time: 4min 44s


In [134]:
# lda_1.print_topics()

In [84]:
lda_1.log_perplexity(corpus[:1000])

-13.003285654860283

Попробуем уменьшить количество тем, но сделать больше проходов:

In [85]:
%%time
lda_2 = gensim.models.LdaMulticore(corpus, 75, id2word=dictionary, passes=15)

Wall time: 5min 54s


In [133]:
# lda_2.print_topics()

In [87]:
lda_2.log_perplexity(corpus[:1000])

-9.71470690425573

Теперь вернемся к параметрам, которые ставили на паре, но изменим параметр alpha на asymmetric:

In [88]:
%%time
lda_3 = gensim.models.LdaMulticore(corpus, 100, id2word=dictionary, passes=10, alpha='asymmetric')

Wall time: 3min 55s
Parser   : 103 ms


In [132]:
# lda_3.print_topics()

In [90]:
lda_3.log_perplexity(corpus[:1000])

-10.906441646396885

В третьей модели с измененной альфой получилось меньше осмысленных тем (хотя они явно есть). А среди первых двух лучшие результаты показывает вторая, где было больше проходов и меньше топиков, перплексия у нее тоже минимальная. Вот удачные темы, которые она выделила:

* 0.060*"дорога" + 0.043*"железнодорожный" + 0.038*"посёлок" + 0.037*"станция" + 0.037*"железный" + 0.032*"километр" + 0.017*"линия" + 0.017*"проходить" + 0.014*"центр" + 0.014*"поезд"')

* 0.023*"партия" + 0.014*"президент" + 0.013*"правительство" + 0.013*"политический" + 0.010*"совет" + 0.010*"организация" + 0.009*"министр" + 0.009*"стать" + 0.009*"страна" + 0.009*"республика"')

* 0.035*"армия" + 0.035*"войско" + 0.029*"фронт" + 0.029*"город" + 0.022*"противник" + 0.018*"наступление" + 0.018*"январь" + 0.017*"район" + 0.016*"часть" + 0.014*"операция"')


Теперь обучим еще три модели с tfidf, в первой оставим "дефолтные" значения с пары, во второй сделаем больше тем, в третьей - больше проходов:

In [92]:
%%time
lda_tf_1 = gensim.models.LdaMulticore(corpus, 100, id2word=dictionary, passes=10)

Wall time: 3min 20s


In [131]:
# lda_tf_1.print_topics()

In [94]:
lda_tf_1.log_perplexity(corpus[:1000])

-25.745204181229308

In [95]:
%%time
lda_tf_2 = gensim.models.LdaMulticore(corpus, 150, id2word=dictionary, passes=10)

Wall time: 3min 2s
Parser   : 168 ms


In [130]:
# lda_tf_2.print_topics()

In [99]:
lda_tf_2.log_perplexity(corpus[:1000])

-53.74885792530168

In [100]:
%%time
lda_tf_3 = gensim.models.LdaMulticore(corpus, 100, id2word=dictionary, passes=15)

Wall time: 4min 38s
Parser   : 102 ms


In [129]:
# lda_tf_3.print_topics()

In [102]:
lda_tf_3.log_perplexity(corpus[:1000])

-28.455099451667664

Лучшая модель, как ни странно, - первая, где минимальное количество тем и проходов. Но все три почему-то получились значительно хуже, чем bow, и на вид, и по перплексии. Хотя иногда в них попадаются интересные топики, которых не встретилось в bow, например, этот:

* 0.077*"граф" + 0.043*"графство" + 0.039*"i" + 0.035*"де" + 0.031*"ii" + 0.031*"герцог" + 0.031*"iii" + 0.026*"сын" + 0.026*"бургундия" + 0.023*"король"')

А вообще темы у моделей lda_2 и lda_tf_1 (которые мы считаем лучшими), кажется, вообще не совпадают, это как-то странно.

# NMF

In [103]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [104]:
stexts = [' '.join(text) for text in texts]

Сделаем матрицу слова-документы с помощью TfidfVectorizer и CountVectorizer с одинаковыми параметрами, потом сравним их между собой.

In [109]:
vectorizer1 = TfidfVectorizer(max_features=2000, min_df=10, max_df=0.3, ngram_range=(1,2))
X1 = vectorizer1.fit_transform(stexts)                    

In [112]:
vectorizer2 = CountVectorizer(max_features=2000, min_df=10, max_df=0.3, ngram_range=(1,2))
X2 = vectorizer2.fit_transform(stexts)

Разложение.

In [114]:
model1 = NMF(n_components=100)
model1.fit(X1)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=100, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [120]:
model2 = NMF(n_components=100)
model2.fit(X2)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=100, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

Отобразим темы первой модели и ее точность:

In [128]:
# feat_names_1 = vectorizer1.get_feature_names()
# top_words = model1.components_.argsort()[:,:-5:-1]

# for i in range(top_words.shape[0]):
#     words = [feat_names_1[j] for j in top_words[i]]
#     print(i, "--".join(words))

In [118]:
model1.reconstruction_err_

1225.116505459658

И то же самое для второй:

In [127]:
# feat_names_2 = vectorizer2.get_feature_names()
# top_words = model2.components_.argsort()[:,:-5:-1]

# for i in range(top_words.shape[0]):
#     words = [feat_names_2[j] for j in top_words[i]]
#     print(i, "--".join(words))

In [122]:
model2.reconstruction_err_

1226.3888384505683

Метрики у них, конечно, совсем большие, но темы выглядят неплохо. Возьмем tfidf-векторайзер и попробуем увеличить количество тем:

In [123]:
model3 = NMF(n_components=200)
model3.fit(X1)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=200, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [126]:
# top_words = model3.components_.argsort()[:,:-5:-1]

# for i in range(top_words.shape[0]):
#     words = [feat_names_1[j] for j in top_words[i]]
#     print(i, "--".join(words))

In [125]:
model3.reconstruction_err_

66.96642475174846

С увеличением количества тем точность стала гораздо выше, так что лучшей считаем последнюю из этих трех моделей. Многие темы у NMF хотя бы частично совпадают с темами из LDA (автомобили, спорт, география). 

Вот некоторые удачные:

* фильм--режиссёр--снять--снятой

* турнир--финал--профессиональный--победитель

* художник--искусство--живопись--выставка

* флаг--символ--цвета--герб

Правда, иногда три слова из четырех образуют тему, а последнее как-то выпадает:

* герой--социалистический--труд--сайт

* римский--папа--католический--китай